In [3]:
import os

In [4]:
%pwd

'c:\\Users\\pc\\Desktop\\SapienText\\SapienText\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\pc\\Desktop\\SapienText\\SapienText'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path
  

In [8]:
from SapienText.constants import *
from SapienText.utils.common import read_yaml, create_directories

In [13]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import urllib.request as request
import zipfile
from SapienText.logging import logger
from SapienText.utils.common import get_size


In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)  # Ensure the directory exists

        print("Downloading from URL:", self.config.source_URL)  # Debug URL
        print("Saving to path:", self.config.local_data_file)  # Debug file path

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! Info: \n{headers}")
        else:
            logger.info("File already exists. Skipping download.")

    # def download_file(self):
    #     if not os.path.exists(self.config.local_data_file):
    #         filename, headers = request.urlretrieve(
    #             url = self.config.source_URL,
    #             filename = self.config.local_data_file
    #         )
    #         logger.info(f"{filename} download! with following info: \n{headers}")
    #     else:
    #         logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-04 12:42:06,443: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-04 12:42:06,444: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-04 12:42:06,446: INFO: common: created directory at: artifacts]
[2025-02-04 12:42:06,447: INFO: common: created directory at: artifacts/data-ingestion]
Saving to path: artifacts/data_ingestion/summarizer-data.zip
[2025-02-04 12:42:10,412: INFO: 2636639979: artifacts/data_ingestion/summarizer-data.zip downloaded! Info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1E8E:12F1B9:89058:D808D:67A1BB4A
Accept-Ranges: bytes
Date: Tue, 04 Feb 2025 07:12:07 GMT
Vi